In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.utils.checkpoint
from ultralytics import YOLO
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.ticker as mtick 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report


import torchvision.transforms as transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision import models
from torchvision.io import read_image
from torchvision.io import write_png
from torchvision.io import ImageReadMode
import torchvision.transforms.functional as transform_fun
from torchvision.datasets import ImageFolder
# from keras.preprocessing.image import ImageDataGenerator
# from keras.preprocessing.image import img_to_array
from torchvision.utils import save_image
from PIL import Image
import glob
import data_loader as DL
import os, shutil
import cv2
import torchvision
import matplotlib.pyplot as plt
import json

In [2]:
device = 'cuda' if torch.cuda.is_available() else  'cpu'
device

0

In [3]:
Npath_NO = 'data/croped/Nomask'
Npath_YES = 'data/croped/masked'
path_labels = 'data/labels/*.txt'
path_img = 'data/images/*.png'

path_train_labels = 'data/train/labels'
path_test_labels = 'data/test/labels'

path_train_img = 'data/train/images'
path_test_img = 'data/test/images'

In [4]:
def img_augment(path, class_prob, dtrain, dtest, label_train_path, label_test_path, Npath):
    if not os.path.exists(label_test_path):
        os.makedirs(label_test_path)
    if not os.path.exists(label_train_path):
        os.makedirs(label_train_path)
    total = len(path)
    print(total)
    i = 1
    aug = torch.nn.Sequential(
        transforms.ColorJitter(brightness=0.5),
        transforms.RandomGrayscale(p=0.2),
        transforms.RandomInvert(p=0.2),
        transforms.RandomEqualize(p=0.2),
        transforms.RandomPosterize(p=0.2,bits=8),
        transforms.RandomAutocontrast(p=0.2),

    )

    transform = transforms.Compose([transforms.ToPILImage()])
    transform2 = transforms.Compose([transforms.ToTensor()])
    list_images_names = list()
    list_images_names_new = list()
    for file in path:
        file_name = file.replace('\\','/').split('/')
        list_images_names.append(file_name[2].replace('.png' , '.txt'))
        for _ in range(0, class_prob):
            img = read_image(path=file, mode=ImageReadMode.RGB)
            img = transform(img)
            X = aug(img)
            save_image(transform2(X).to(torch.float),fp=Npath+'/'+ f'mask{total+i}.png')
            list_images_names_new.append(f'mask{total+i}.txt')
            i+=1
    
    
    # add labels
    for img, New_img in zip(list_images_names, list_images_names_new):
        
        df_train = dtrain.loc[dtrain['file_name'] == img]
        df_test = dtest.loc[dtest['file_name'] == img]
        
        list_nums_train = df_train.drop(['file_name'],axis=1).values
        list_nums_test = df_test.drop(['file_name'],axis=1).values
        
        for val_train, val_test in zip(list_nums_train, list_nums_test):
            with open(label_train_path + '/' + New_img, 'a') as f_train:
                   f_train.write(str(int(val_train[0]))+ ' '+ str(val_train[1])+ ' '+ str(val_train[2])+ ' '+ str(val_train[3])+ ' '+ str(val_train[4])+ '\n')
            f_train.close()
            
            with open(label_test_path + '/' + New_img, 'a') as f_test:
                   f_test.write(str(int(val_test[0]))+ ' '+ str(val_test[1])+ ' '+ str(val_test[2])+ ' '+ str(val_test[3])+ ' '+ str(val_test[4])+ '\n')
            f_test.close()

In [5]:
def img_augment_(img_path, class_prob, df, label_path):
    
    df2 = df['file_name'].value_counts()
    
    list_images_names = list()
    list_images_names_new = list()
    
    aug = torch.nn.Sequential(
        transforms.ColorJitter(brightness=0.5),
        transforms.RandomGrayscale(p=0.2),
        transforms.RandomInvert(p=0.2),
        transforms.RandomEqualize(p=0.2),
        transforms.RandomPosterize(p=0.2,bits=8),
        transforms.RandomAutocontrast(p=0.2),

    )
    
    transform = transforms.Compose([transforms.ToPILImage()])
    transform2 = transforms.Compose([transforms.ToTensor()])
    
    if not os.path.exists(label_path):
        os.makedirs(label_path)
    
    for count, img_name in zip(df2, df2.index):
        img_name = img_name.replace('.txt', '')
        
        for i in range(0 , class_prob):
            img = read_image(path=os.path.join(img_path, img_name + '.png'), mode=ImageReadMode.RGB)
            img = transform(img)
            img = aug(img)
            save_image(transform2(img).to(torch.float),fp=os.path.join(img_path, img_name + f'_{str(i)}_augmented_.png'))
            list_images_names_new.append(img_name  + f'_{str(i)}_augmented_.txt')
            list_images_names.append(img_name +'.txt')
        print(list_images_names_new, len(list_images_names_new))
    
#     # add labels
    for img, New_img in zip(list_images_names, list_images_names_new):
        
        df2 = df.loc[df['file_name'] == img]
        
        list_values = df2.drop(['file_name'],axis=1).values
        
        for value in list_values:
            with open(label_path + '/' + New_img, 'a') as f:
                   f.write(str(int(value[0]))+ ' '+ str(value[1])+ ' '+ str(value[2])+ ' '+ str(value[3])+ ' '+ str(value[4])+ '\n')
            f.close()
            

In [6]:
def change_8bit(paths):
    transform = transforms.Compose([transforms.ToPILImage()])
    transform2 = transforms.Compose([transforms.ToTensor()])
    for path in paths:
        img = cv2.imread(path, cv2.IMREAD_UNCHANGED)

        img = cv2.normalize(img,None,0,255, cv2.NORM_MINMAX, dtype= cv2.CV_8U)
#             img = transform(img)
        cv2.imwrite(path, img)
    

In [7]:
def Txt_toPandas(file_paths):
    list_nums = list()
    for file in file_paths:
        with open(file) as f:
            lines = f.readlines()
            for line in lines:
                a = str(line).split()
                file_name = file.replace('\\','/').split('/')
                list_nums.append([int(a[0]),float(a[1]),float(a[2]),float(a[3]),float(a[4]), file_name[2]])
            f.close()
    return pd.DataFrame(list_nums, columns=['label', 'x_center', 'y_center', 'width', 'height', 'file_name'])

In [8]:
def Pandas_toTxt(path, df):
    if not os.path.exists(path):
        os.makedirs(path)
        
    list_file_names = df['file_name'].values
    list_nums = df.drop(['file_name'],axis=1).values
    for values, file in zip(list_nums, list_file_names):
        with open(path + '/' + file, 'a') as f:
            f.write(str(int(values[0]))+ ' '+ str(values[1])+ ' '+ str(values[2])+ ' '+ str(values[3])+ ' '+ str(values[4])+ '\n')
            f.close()

In [9]:
def Move_img(img_path, label_path, file_path):
    img_name = glob.glob(label_path + '/*.txt')

    if not os.path.exists(file_path):
        os.makedirs(file_path)
    
    for file in img_name:
        img = file.replace('.txt', '.png').split('\\')
        shutil.copy2(os.path.join(img_path, img[1]), file_path)
    

In [10]:
dt = Txt_toPandas(glob.glob(path_labels))
dt

,label,x_center,y_center,width,height,file_name
0,0,0.383124,0.296478,0.171129,0.241414,mask01.txt
1,1,0.648483,0.273751,0.207955,0.309091,mask01.txt
2,1,0.170476,0.319206,0.124195,0.153535,mask01.txt
3,1,0.934781,0.491428,0.075817,0.122222,mask01.txt
4,0,0.613534,0.294497,0.142211,0.411052,mask02.txt
...,...,...,...,...,...,...
1792,1,0.203744,0.584862,0.152727,0.262626,mask97.txt
1793,0,0.671159,0.219711,0.157052,0.144444,mask98.txt
1794,0,0.547203,0.204559,0.157773,0.261616,mask99.txt
1795,0,0.895112,0.300519,0.152379,0.219192,mask99.txt


In [13]:
change_8bit(glob.glob(path_img))

In [14]:
a ,b = dt.label.value_counts()
a, b

(1437, 360)

In [15]:
# dt['label_name'] = dt['label_name'].replace({'mask': 1, 'No_mask': 0})
y = dt['label']
X = dt.drop(['label'], axis=1)

### Train test split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state = 42)

In [27]:
train = pd.concat([y_train, X_train],axis=1)
train_No_mask = train.loc[train.label == 1]
test = pd.concat([y_test, X_test], axis = 1)
test_No_mask = test.loc[test.label == 1]
train_No_mask

img_augment_(path_img.replace('*.png', ''), 4, train_No_mask, path_train_labels)
img_augment_(path_img.replace('*.png', ''), 4, test_No_mask, path_test_labels)
# img_augment(glob.glob(path_img), 4, train_No_mask, test_No_mask, path_train_labels, path_test_labels, path_img.replace('*.png', ''))

['mask241_0_augmented_.txt', 'mask241_1_augmented_.txt', 'mask241_2_augmented_.txt', 'mask241_3_augmented_.txt'] 4
['mask241_0_augmented_.txt', 'mask241_1_augmented_.txt', 'mask241_2_augmented_.txt', 'mask241_3_augmented_.txt', 'mask111_0_augmented_.txt', 'mask111_1_augmented_.txt', 'mask111_2_augmented_.txt', 'mask111_3_augmented_.txt'] 8
['mask241_0_augmented_.txt', 'mask241_1_augmented_.txt', 'mask241_2_augmented_.txt', 'mask241_3_augmented_.txt', 'mask111_0_augmented_.txt', 'mask111_1_augmented_.txt', 'mask111_2_augmented_.txt', 'mask111_3_augmented_.txt', 'mask297_0_augmented_.txt', 'mask297_1_augmented_.txt', 'mask297_2_augmented_.txt', 'mask297_3_augmented_.txt'] 12
['mask241_0_augmented_.txt', 'mask241_1_augmented_.txt', 'mask241_2_augmented_.txt', 'mask241_3_augmented_.txt', 'mask111_0_augmented_.txt', 'mask111_1_augmented_.txt', 'mask111_2_augmented_.txt', 'mask111_3_augmented_.txt', 'mask297_0_augmented_.txt', 'mask297_1_augmented_.txt', 'mask297_2_augmented_.txt', 'mask297_

['mask241_0_augmented_.txt', 'mask241_1_augmented_.txt', 'mask241_2_augmented_.txt', 'mask241_3_augmented_.txt', 'mask111_0_augmented_.txt', 'mask111_1_augmented_.txt', 'mask111_2_augmented_.txt', 'mask111_3_augmented_.txt', 'mask297_0_augmented_.txt', 'mask297_1_augmented_.txt', 'mask297_2_augmented_.txt', 'mask297_3_augmented_.txt', 'mask306_0_augmented_.txt', 'mask306_1_augmented_.txt', 'mask306_2_augmented_.txt', 'mask306_3_augmented_.txt', 'mask210_0_augmented_.txt', 'mask210_1_augmented_.txt', 'mask210_2_augmented_.txt', 'mask210_3_augmented_.txt', 'mask53_0_augmented_.txt', 'mask53_1_augmented_.txt', 'mask53_2_augmented_.txt', 'mask53_3_augmented_.txt', 'mask105_0_augmented_.txt', 'mask105_1_augmented_.txt', 'mask105_2_augmented_.txt', 'mask105_3_augmented_.txt', 'mask122_0_augmented_.txt', 'mask122_1_augmented_.txt', 'mask122_2_augmented_.txt', 'mask122_3_augmented_.txt', 'mask157_0_augmented_.txt', 'mask157_1_augmented_.txt', 'mask157_2_augmented_.txt', 'mask157_3_augmented_.t

['mask241_0_augmented_.txt', 'mask241_1_augmented_.txt', 'mask241_2_augmented_.txt', 'mask241_3_augmented_.txt', 'mask111_0_augmented_.txt', 'mask111_1_augmented_.txt', 'mask111_2_augmented_.txt', 'mask111_3_augmented_.txt', 'mask297_0_augmented_.txt', 'mask297_1_augmented_.txt', 'mask297_2_augmented_.txt', 'mask297_3_augmented_.txt', 'mask306_0_augmented_.txt', 'mask306_1_augmented_.txt', 'mask306_2_augmented_.txt', 'mask306_3_augmented_.txt', 'mask210_0_augmented_.txt', 'mask210_1_augmented_.txt', 'mask210_2_augmented_.txt', 'mask210_3_augmented_.txt', 'mask53_0_augmented_.txt', 'mask53_1_augmented_.txt', 'mask53_2_augmented_.txt', 'mask53_3_augmented_.txt', 'mask105_0_augmented_.txt', 'mask105_1_augmented_.txt', 'mask105_2_augmented_.txt', 'mask105_3_augmented_.txt', 'mask122_0_augmented_.txt', 'mask122_1_augmented_.txt', 'mask122_2_augmented_.txt', 'mask122_3_augmented_.txt', 'mask157_0_augmented_.txt', 'mask157_1_augmented_.txt', 'mask157_2_augmented_.txt', 'mask157_3_augmented_.t

['mask241_0_augmented_.txt', 'mask241_1_augmented_.txt', 'mask241_2_augmented_.txt', 'mask241_3_augmented_.txt', 'mask111_0_augmented_.txt', 'mask111_1_augmented_.txt', 'mask111_2_augmented_.txt', 'mask111_3_augmented_.txt', 'mask297_0_augmented_.txt', 'mask297_1_augmented_.txt', 'mask297_2_augmented_.txt', 'mask297_3_augmented_.txt', 'mask306_0_augmented_.txt', 'mask306_1_augmented_.txt', 'mask306_2_augmented_.txt', 'mask306_3_augmented_.txt', 'mask210_0_augmented_.txt', 'mask210_1_augmented_.txt', 'mask210_2_augmented_.txt', 'mask210_3_augmented_.txt', 'mask53_0_augmented_.txt', 'mask53_1_augmented_.txt', 'mask53_2_augmented_.txt', 'mask53_3_augmented_.txt', 'mask105_0_augmented_.txt', 'mask105_1_augmented_.txt', 'mask105_2_augmented_.txt', 'mask105_3_augmented_.txt', 'mask122_0_augmented_.txt', 'mask122_1_augmented_.txt', 'mask122_2_augmented_.txt', 'mask122_3_augmented_.txt', 'mask157_0_augmented_.txt', 'mask157_1_augmented_.txt', 'mask157_2_augmented_.txt', 'mask157_3_augmented_.t

['mask241_0_augmented_.txt', 'mask241_1_augmented_.txt', 'mask241_2_augmented_.txt', 'mask241_3_augmented_.txt', 'mask111_0_augmented_.txt', 'mask111_1_augmented_.txt', 'mask111_2_augmented_.txt', 'mask111_3_augmented_.txt', 'mask297_0_augmented_.txt', 'mask297_1_augmented_.txt', 'mask297_2_augmented_.txt', 'mask297_3_augmented_.txt', 'mask306_0_augmented_.txt', 'mask306_1_augmented_.txt', 'mask306_2_augmented_.txt', 'mask306_3_augmented_.txt', 'mask210_0_augmented_.txt', 'mask210_1_augmented_.txt', 'mask210_2_augmented_.txt', 'mask210_3_augmented_.txt', 'mask53_0_augmented_.txt', 'mask53_1_augmented_.txt', 'mask53_2_augmented_.txt', 'mask53_3_augmented_.txt', 'mask105_0_augmented_.txt', 'mask105_1_augmented_.txt', 'mask105_2_augmented_.txt', 'mask105_3_augmented_.txt', 'mask122_0_augmented_.txt', 'mask122_1_augmented_.txt', 'mask122_2_augmented_.txt', 'mask122_3_augmented_.txt', 'mask157_0_augmented_.txt', 'mask157_1_augmented_.txt', 'mask157_2_augmented_.txt', 'mask157_3_augmented_.t

...

['mask241_0_augmented_.txt', 'mask241_1_augmented_.txt', 'mask241_2_augmented_.txt', 'mask241_3_augmented_.txt', 'mask210_0_augmented_.txt', 'mask210_1_augmented_.txt', 'mask210_2_augmented_.txt', 'mask210_3_augmented_.txt', 'mask389_0_augmented_.txt', 'mask389_1_augmented_.txt', 'mask389_2_augmented_.txt', 'mask389_3_augmented_.txt', 'mask177_0_augmented_.txt', 'mask177_1_augmented_.txt', 'mask177_2_augmented_.txt', 'mask177_3_augmented_.txt', 'mask224_0_augmented_.txt', 'mask224_1_augmented_.txt', 'mask224_2_augmented_.txt', 'mask224_3_augmented_.txt', 'mask138_0_augmented_.txt', 'mask138_1_augmented_.txt', 'mask138_2_augmented_.txt', 'mask138_3_augmented_.txt', 'mask06_0_augmented_.txt', 'mask06_1_augmented_.txt', 'mask06_2_augmented_.txt', 'mask06_3_augmented_.txt', 'mask190_0_augmented_.txt', 'mask190_1_augmented_.txt', 'mask190_2_augmented_.txt', 'mask190_3_augmented_.txt', 'mask319_0_augmented_.txt', 'mask319_1_augmented_.txt', 'mask319_2_augmented_.txt', 'mask319_3_augmented_.t

['mask241_0_augmented_.txt', 'mask241_1_augmented_.txt', 'mask241_2_augmented_.txt', 'mask241_3_augmented_.txt', 'mask210_0_augmented_.txt', 'mask210_1_augmented_.txt', 'mask210_2_augmented_.txt', 'mask210_3_augmented_.txt', 'mask389_0_augmented_.txt', 'mask389_1_augmented_.txt', 'mask389_2_augmented_.txt', 'mask389_3_augmented_.txt', 'mask177_0_augmented_.txt', 'mask177_1_augmented_.txt', 'mask177_2_augmented_.txt', 'mask177_3_augmented_.txt', 'mask224_0_augmented_.txt', 'mask224_1_augmented_.txt', 'mask224_2_augmented_.txt', 'mask224_3_augmented_.txt', 'mask138_0_augmented_.txt', 'mask138_1_augmented_.txt', 'mask138_2_augmented_.txt', 'mask138_3_augmented_.txt', 'mask06_0_augmented_.txt', 'mask06_1_augmented_.txt', 'mask06_2_augmented_.txt', 'mask06_3_augmented_.txt', 'mask190_0_augmented_.txt', 'mask190_1_augmented_.txt', 'mask190_2_augmented_.txt', 'mask190_3_augmented_.txt', 'mask319_0_augmented_.txt', 'mask319_1_augmented_.txt', 'mask319_2_augmented_.txt', 'mask319_3_augmented_.t

['mask241_0_augmented_.txt', 'mask241_1_augmented_.txt', 'mask241_2_augmented_.txt', 'mask241_3_augmented_.txt', 'mask210_0_augmented_.txt', 'mask210_1_augmented_.txt', 'mask210_2_augmented_.txt', 'mask210_3_augmented_.txt', 'mask389_0_augmented_.txt', 'mask389_1_augmented_.txt', 'mask389_2_augmented_.txt', 'mask389_3_augmented_.txt', 'mask177_0_augmented_.txt', 'mask177_1_augmented_.txt', 'mask177_2_augmented_.txt', 'mask177_3_augmented_.txt', 'mask224_0_augmented_.txt', 'mask224_1_augmented_.txt', 'mask224_2_augmented_.txt', 'mask224_3_augmented_.txt', 'mask138_0_augmented_.txt', 'mask138_1_augmented_.txt', 'mask138_2_augmented_.txt', 'mask138_3_augmented_.txt', 'mask06_0_augmented_.txt', 'mask06_1_augmented_.txt', 'mask06_2_augmented_.txt', 'mask06_3_augmented_.txt', 'mask190_0_augmented_.txt', 'mask190_1_augmented_.txt', 'mask190_2_augmented_.txt', 'mask190_3_augmented_.txt', 'mask319_0_augmented_.txt', 'mask319_1_augmented_.txt', 'mask319_2_augmented_.txt', 'mask319_3_augmented_.t

['mask241_0_augmented_.txt', 'mask241_1_augmented_.txt', 'mask241_2_augmented_.txt', 'mask241_3_augmented_.txt', 'mask210_0_augmented_.txt', 'mask210_1_augmented_.txt', 'mask210_2_augmented_.txt', 'mask210_3_augmented_.txt', 'mask389_0_augmented_.txt', 'mask389_1_augmented_.txt', 'mask389_2_augmented_.txt', 'mask389_3_augmented_.txt', 'mask177_0_augmented_.txt', 'mask177_1_augmented_.txt', 'mask177_2_augmented_.txt', 'mask177_3_augmented_.txt', 'mask224_0_augmented_.txt', 'mask224_1_augmented_.txt', 'mask224_2_augmented_.txt', 'mask224_3_augmented_.txt', 'mask138_0_augmented_.txt', 'mask138_1_augmented_.txt', 'mask138_2_augmented_.txt', 'mask138_3_augmented_.txt', 'mask06_0_augmented_.txt', 'mask06_1_augmented_.txt', 'mask06_2_augmented_.txt', 'mask06_3_augmented_.txt', 'mask190_0_augmented_.txt', 'mask190_1_augmented_.txt', 'mask190_2_augmented_.txt', 'mask190_3_augmented_.txt', 'mask319_0_augmented_.txt', 'mask319_1_augmented_.txt', 'mask319_2_augmented_.txt', 'mask319_3_augmented_.t

['mask241_0_augmented_.txt', 'mask241_1_augmented_.txt', 'mask241_2_augmented_.txt', 'mask241_3_augmented_.txt', 'mask210_0_augmented_.txt', 'mask210_1_augmented_.txt', 'mask210_2_augmented_.txt', 'mask210_3_augmented_.txt', 'mask389_0_augmented_.txt', 'mask389_1_augmented_.txt', 'mask389_2_augmented_.txt', 'mask389_3_augmented_.txt', 'mask177_0_augmented_.txt', 'mask177_1_augmented_.txt', 'mask177_2_augmented_.txt', 'mask177_3_augmented_.txt', 'mask224_0_augmented_.txt', 'mask224_1_augmented_.txt', 'mask224_2_augmented_.txt', 'mask224_3_augmented_.txt', 'mask138_0_augmented_.txt', 'mask138_1_augmented_.txt', 'mask138_2_augmented_.txt', 'mask138_3_augmented_.txt', 'mask06_0_augmented_.txt', 'mask06_1_augmented_.txt', 'mask06_2_augmented_.txt', 'mask06_3_augmented_.txt', 'mask190_0_augmented_.txt', 'mask190_1_augmented_.txt', 'mask190_2_augmented_.txt', 'mask190_3_augmented_.txt', 'mask319_0_augmented_.txt', 'mask319_1_augmented_.txt', 'mask319_2_augmented_.txt', 'mask319_3_augmented_.t

In [28]:
Pandas_toTxt(path_train_labels, train)
Pandas_toTxt(path_test_labels, test)


In [30]:
Move_img(path_img.replace('*.png' , ''), path_train_labels, path_train_img)
Move_img(path_img.replace('*.png' , ''), path_test_labels, path_test_img)

## training


In [ ]:
model = YOLO('yolov8x.pt')

results = model.train(
    data='data.yaml',
    imgsz=640,
    epochs=12,
    batch=5,
    workers=1)

In [ ]:
results = model.val(
    data='data.yaml',
    imgsz=640,
    epochs=10,
    batch=8,
    workers=1)

In [12]:
cap = cv2.VideoCapture(0)

In [ ]:
from ultralytics.yolo.utils.plotting import Annotator



while True:
    _, frame = cap.read()
    
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = model.predict(img)

    for r in results:
        
        annotator = Annotator(frame)
        
        boxes = r.boxes
        for box in boxes:
            
            b = box.xyxy[0]  # get box coordinates in (top, left, bottom, right) format
            c = box.cls
            annotator.box_label(b, model.names[int(c)])
          
    frame = annotator.result()  
    cv2.imshow('YOLO V8 Detection', frame)     
    if cv2.waitKey(1) & 0xFF == ord(' '):
        break

cap.release()
cv2.destroyAllWindows()
